# Import github directory

In [1]:
!rm -rf *
!git clone https://github.com/marcocuccarini/ReCLAIMProject.git; mv ReCLAIMProject/* .; rm -rf ReCLAIMProject;

Cloning into 'ReCLAIMProject'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 1), reused 5 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), 106.14 KiB | 35.38 MiB/s, done.
Resolving deltas: 100% (1/1), done.


#Install libraries

In [2]:
!pip install torch==2.3.1
!pip install transformers==4.41.2
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git@64bb8cfd512a9dcd860d21563b624676f7432ec5"
!pip install --no-deps "xformers==0.0.26.post1" "trl<0.9.0" peft==0.11.1 accelerate==0.32.0 bitsandbytes==0.43.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#Import package

In [3]:
import pandas as pd
import numpy
from transformers import GenerationConfig
from unsloth import FastLanguageModel
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support, classification_report
import matplotlib.pyplot as plt
from sklearn import metrics

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


#Import dataset

In [4]:
df=pd.read_csv('/content/Data_Reapropriation.csv', sep=";")

#LLM hyerparameters

In [5]:
max_seq_length = 2048 # Choose any! Unsloth supports RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
temperature=1

#List of prompts

In [6]:
Prompt1="Stabilisci se la frase contiene riappropriazione semantica, rispondi 'Vero' se lo contiene e 'Falso' altrimenti."
Prompt2="Stabilisci se la frase contiene un linguaggio che non ha intenzione di offendere, Rispondi 'Vero' se lo contiene e 'Falso' altrimenti."
Prompt3="Stabilisci se la frase contiene un linguaggio che intende offendere delle persone per il loro orientamento sessuale e le loro identità di genere, rispondi 'Vero' se lo contiene e 'Falso' altrimenti."
Prompt4="Stabilisci se nella frase proposta le seguenti parole “frocio, invertito, travestit*, checca, deviato, culattone, finocchio, finocchi, omosex, finocchietto, omosessuali, frocia, ricchione, trans, troia” sono utilizzate per offendere le persone per il loro orientamento sessuale e/o identità di genere. Rispondi 'Vero' se c’è un intento offensivo, altrimenti 'Falso'."

# LLM instantiation

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2-7B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    temperature=temperature
)

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Qwen2 patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Prompt1 inference sample

In [ ]:
prompt = """
Instruzioni:
{}
Testo:
{}

Answer:
"""
sys_ans = []
for index, row in df.iterrows():
  messages = [
      {"role": "system", "content": "Sei un assistente di lingua italiana. Di seguito è riportata un'istruzione che descrive un compito, accompagnata da un input che fornisce un ulteriore contesto. Scrivete una risposta che completi in modo appropriato la richiesta."},
      {"role": "user", "content": prompt.format(Prompt1,row['text'])}
  ]
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

  # Directly use generate() and tokenizer.decode() to get the output.
  # Use `max_new_tokens` to control the maximum output length.
  generated_ids = model.generate(
      model_inputs.input_ids,
      max_new_tokens=512,
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  sys_ans.append(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])


#Different answers provide by the model

In [ ]:
set(sys_ans)

#Classification report

In [ ]:
pred_label = [1 if item[:4] == 'Vero' or  item[:14] == 'Risposta: Vero' else 0 for item in sys_ans]
true_label = df['reappropriation'].tolist()

# Compute precision, recall, and f1-score
print(classification_report(gold_ans,sys_ans1))

#

In [ ]:
actual = numpy.array(true_label)
predicted = numpy.array(pred_label)

confusion_matrix = metrics.confusion_matrix(actual, predicted)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [0, 1])

cm_display.plot()
plt.show()